In [ ]:
import cv2
import numpy as np
import scipy.linalg as la

# For profiling, not required in general.
import cProfile

In [11]:
def frame_distance(frame1, frame2, stride:int = 20):

    frame1_strided = frame1[::stride, ::stride, :].reshape(-1 , 3).astype(int)
    frame2_strided = frame2[::stride, ::stride, :].reshape(-1 , 3).astype(int)

    x = la.norm(
        frame1_strided - frame2_strided,
        ord=2,
        axis=1
        ).mean()

    return x

In [ ]:
def find_cuts(file_path, stride:int = 20):

    # Start the video capture
    vc = cv2.VideoCapture(file_path)

    # Get total frames
    total_frames = vc.get(cv2.CAP_PROP_FRAME_COUNT)
    print(f"Total frames: {total_frames}")

    curr_frame = None
    prev_frame = None
    frame_diffs = np.zeros(int(total_frames - 1))
    idx = 0

    while vc.isOpened():

        ret, frame = vc.read()

        if ret:

            if curr_frame is None:
                curr_frame = frame
            else:
                prev_frame = curr_frame
                curr_frame = frame

                frame_diffs[idx] = frame_distance(curr_frame, prev_frame, stride=stride)
                idx += 1

        else:
            vc.release()
            break

    return frame_diffs

In [ ]:
find_cuts("video/2021-09-16_16-16-44.mp4", stride=50)

In [ ]:
# If you want to see the profiling results
cProfile.run('find_cuts("video/2021-09-16_16-16-44.mp4", stride=50)')